In [1]:
!git pull

import pandas as pd 
import jsonlines 
import json 
import lmdb 
import os
import lmdb
from activation_logging.activations_logger import ActivationsLogger
from activation_logging.activation_parser import ActivationParser 



Already up to date.


/home/hy3134/notebook_llm/HalluLens/activation_logging/activations_logger.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

inference_json = 'shared/goodwiki_jsonv2/generation.jsonl'
eval_json='shared/goodwiki_jsonv2/eval_results.json'     # JSON format
activations_path='shared/goodwiki_jsonv2/activations.json'

ap = ActivationParser(
    inference_json=inference_json,  # JSONL format
    eval_json=eval_json,      # JSON format
    activations_path=activations_path,     # Directory path
    logger_type="json"                                      # JSON logger type
)



# ap = ActivationParser(infer_json, eval_json, lmdb_path)
relevant_layers = [i for i in range(14,30)]
target_layers = [22, 26]

outlier_class = 1
fixed_layer = None

train_dataset = ap.get_dataset('train', 
                               relevant_layers = relevant_layers,
                              )

2026-01-20 10:54:52.868 | INFO     | activation_logging.activations_logger:__init__:595 - JsonActivationsLogger.__init__ starting - output_dir: shared/goodwiki_jsonv2/activations.json, target_layers: all, sequence_mode: all, read_only: True
2026-01-20 10:54:52.869 | INFO     | activation_logging.activations_logger:__init__:603 - JsonActivationsLogger.__init__ - Setting up paths
2026-01-20 10:54:52.869 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 10:54:52.871 | INFO     | activation_logging.activations_logger:__init__:638 - JsonActivationsLogger.__init__ - Legacy metadata file exists, loading...
2026-01-20 10:55:00.359 | INFO     | activation_logging.activations_logger:__init__:644 - JsonActivationsLogger.__init__ - Legacy metadata loaded successfully
2026-01-20 10:55:00.361 | INFO     | activation_logging.activations_logger:__init__:665 - JsonActivationsLogger (Index+Sidecar format) initialized at shared/

In [3]:
len(ap.df[~ap.df['halu']])

6931

In [4]:


train_dataset = ap.get_dataset('train', 
                               relevant_layers = relevant_layers,
                              )
test_dataset = ap.get_dataset('test', relevant_layers = relevant_layers)

print(len(train_dataset))
print(len(test_dataset))

from loguru import logger
import sys 
from activation_research.model import ProgressiveCompressor, LastLayerHaluClassifier, HallucinationClassifier
from activation_research.training import train_contrastive, train_halu_classifier



logger.remove()           # Remove default handler
logger.add(sys.stdout, level="INFO")  # Add handler that logs INFO+





# device = 'cuda'
# # Initialize your model
# # model = LastLayerHaluClassifier(input_dim=4096)

# model = HallucinationClassifier(dim = 4096, layer_index = 28)

# # Train with CUDA (if available)
# train_halu_classifier(model, train_dataset, test_dataset=test_dataset,
#                  epochs=10, batch_size=512, lr=1e-5, device=device, num_workers=8,
#                   sub_batch_size=64, persistent_workers = True, 
                 
#                  )



38159
9540


1

In [6]:
import glob 

glob.glob('checkpoints/*')


['checkpoints/contrastive_checkpoint_epoch_15.pt',
 'checkpoints/contrastive_best.pt',
 'checkpoints/contrastive_checkpoint_epoch_135.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_45.pt',
 'checkpoints/contrastive_checkpoint_epoch_110.pt',
 'checkpoints/contrastive_checkpoint_epoch_10.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_60.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_100.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_65.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_70.pt',
 'checkpoints/contrastive_checkpoint_epoch_90.pt',
 'checkpoints/contrastive_checkpoint_epoch_35.pt',
 'checkpoints/contrastive_checkpoint_epoch_55.pt',
 'checkpoints/contrastive_checkpoint_epoch_130.pt',
 'checkpoints/contrastive_checkpoint_epoch_60.pt',
 'checkpoints/contrastive_checkpoint_epoch_65.pt',
 'checkpoints/contrastive_checkpoint_epoch_30.pt',
 'checkpoints/halu_classifier_checkpoint_epoch_5.pt',
 'checkpoints/contrastive_checkpoint_epoch_125.pt',
 'checkpoints/halu

In [ ]:
from loguru import logger
import sys 
from activation_research.model import ProgressiveCompressor
from activation_research.training import train_contrastive
from activation_research.evaluation import mahalanobis_ood_stats_multilayer
import torch


logger.remove()           # Remove default handler
logger.add(sys.stdout, level="INFO")  # Add handler that logs INFO+

logger.add("logfiledebug.log", rotation="10 MB", retention="7 days")
logger.add("logfile.log", rotation="10 MB", retention="7 days", level = 'INFO')


device = 'cuda'
# Initialize your model
model = ProgressiveCompressor(input_dim=4096, final_dim=512, input_dropout = 0.3)

# model.load_state_dict(torch.load("./checkpoints/contrastive_checkpoint_epoch_10.pt")['model_state_dict'])



# Train with CUDA (if available)
train_contrastive(model, train_dataset, test_dataset=test_dataset,
                 epochs=150, batch_size=512, lr=1e-5, temperature=0.25, device=device, 
                  num_workers=16, sub_batch_size=64, use_labels = True, ignore_label = outlier_class, 
resume_from='contrastive_checkpoint_epoch_100.pt'
                 )

# model.load_state_dict(torch.load('checkpoints/contrastive_checkpoint_epoch_100.pt')['model_state_dict'])

model = model.cuda()


2026-01-20 13:16:38.716 | INFO     | activation_research.training:train_contrastive:207 - Resumed training from epoch 100


Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

2026-01-20 13:16:39.347 | INFO     | activation_research.training:train_contrastive:231 - Starting epoch 101/150



Epoch 101/150:   0%|          | 0/597 [00:00<?, ?it/s]

2026-01-20 13:16:40.961 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.961 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.958 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.956 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.958 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.959 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.959 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 13:16:40.

...

2026-01-20 13:20:29.887 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 360/597



Epoch 101/150:  60%|██████    | 360/597 [03:52<02:44,  1.44it/s, cosine_sim=0.992, loss=3.64, pairing_acc=0.995]

2026-01-20 13:20:31.783 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 361/597
2026-01-20 13:20:31.790 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 362/597
2026-01-20 13:20:31.796 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 363/597
2026-01-20 13:20:31.802 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 364/597
2026-01-20 13:20:31.809 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 365/597
2026-01-20 13:20:31.815 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 366/597
2026-01-20 13:20:31.822 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 367/597
2026-01-20 13:20:31.828 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 368/597



Epoch 101/150:  62%|██████▏   | 368/597 [03:54<01:39,  2.31it/s, cosine_sim=0.992, loss=3.64, pairing_acc=0.995]

2026-01-20 13:20:33.691 | INFO     | activation_research.training:train_contrastive:247 - Epoch 101/150 - Step 369/597


In [12]:
# len(eval_dataset)
from tqdm import tqdm 
from torch.utils.data import DataLoader

def inference_embeddings(model, dataset, batch_size=512, sub_batch_size=64, device='cuda', num_workers=4, layers=None, persistent_workers=False):
    assert batch_size % sub_batch_size == 0, "batch_size must be divisible by sub_batch_size"

    model = model.to(device)
    model.eval()

    dataloader = DataLoader(
        dataset,
        batch_size=sub_batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=persistent_workers
    )

    buffer_x1, buffer_x2, buffer_hash = [], [], []
    if layers is not None:
        buffer_layers = {layer_idx: [] for layer_idx in layers}
    results = []

    subs_in_batch = batch_size // sub_batch_size

    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader, desc="Inference")):
            if layers is None:
                x1 = batch['layer1_activations'].squeeze(1).to(device, non_blocking=True)
                x2 = batch['layer2_activations'].squeeze(1).to(device, non_blocking=True)
                buffer_x1.append(x1)
                buffer_x2.append(x2)
            else:
                all_activations = batch['all_activations']
                for layer_idx, layer_acts in zip(layers, all_activations):
                    buffer_layers[layer_idx].append(layer_acts.squeeze(1).to(device, non_blocking=True))
            
            hashkeys = batch['hashkey']
            buffer_hash.extend(hashkeys)

            # Process when buffer is full or at the end of the loop
            if (layers is None and len(buffer_x1) == subs_in_batch) or \
               (layers is not None and len(next(iter(buffer_layers.values()))) == subs_in_batch) or \
               i == len(dataloader) - 1:
                
                if layers is None:
                    x1_full = torch.cat(buffer_x1, dim=0)
                    x2_full = torch.cat(buffer_x2, dim=0)
                    buffer_x1, buffer_x2 = [], []

                    z1 = model(x1_full)
                    z2 = model(x2_full)

                    for h, z1_i, z2_i in zip(buffer_hash, z1, z2):
                        results.append({
                            "hashkey": h,
                            "z1": z1_i.cpu(),
                            "z2": z2_i.cpu()
                        })
                else:
                    layer_embeddings = {}
                    for layer_idx in layers:
                        layer_buffer = buffer_layers[layer_idx]
                        layer_full = torch.cat(layer_buffer, dim=0)
                        z = model(layer_full)
                        layer_embeddings[f"layer_{layer_idx}"] = z.cpu()
                        buffer_layers[layer_idx] = []

                    # Consistency check
                    n_hash = len(buffer_hash)
                    n_emb = next(iter(layer_embeddings.values())).shape[0]
                    assert n_hash == n_emb, f"buffer_hash length {n_hash} does not match embeddings batch size {n_emb}"
                    for idx, h in enumerate(buffer_hash):
                        results.append({
                            "hashkey": h,
                            "layer_embeddings": {k: v[idx] for k, v in layer_embeddings.items()}
                        })

                buffer_hash = []

    return results
    

In [20]:

ap = ActivationParser(
    inference_json=inference_json,  # JSONL format
    eval_json=eval_json,      # JSON format
    activations_path=activations_path,     # Directory path
    logger_type="json"                                      # JSON logger type
)


if outlier_class == 0: 
    ap.df = ap.df[ap.df['halu']] 
elif outlier_class == 1:
    ap.df = ap.df[~ap.df['halu']]

train_dataset_for_inference = ap.get_dataset('train', 
                               relevant_layers = target_layers,
                                     fixed_layer = fixed_layer
                                
                              )

ap_full = ActivationParser(
    inference_json=inference_json,  # JSONL format
    eval_json=eval_json,      # JSON format
    activations_path=activations_path,     # Directory path
    logger_type="json"                                      # JSON logger type
)



train_layers = None

baseline_embeddings = inference_embeddings(
    model,
    train_dataset_for_inference,
    batch_size=256,
    sub_batch_size=64,
    device='cuda',
    num_workers=8,
    layers = train_layers,
    persistent_workers = False
)

eval_dataset = ap_full.get_dataset('test', relevant_layers = target_layers,
                                  fixed_layer = fixed_layer)

embeddings = inference_embeddings(
    model,
    eval_dataset,
    batch_size=256,
    sub_batch_size=64,
    device='cuda',
    num_workers=8,
    layers = train_layers,
    persistent_workers = False
)

res = embeddings 
df = ap_full.df.head(10000000)

for i, r in enumerate(res): 

    ishalu = df[df['prompt_hash'] == r['hashkey']]['halu']
    assert len(ishalu) == 1 
    res[i]['halu'] = ishalu.values


2026-01-20 12:46:35.090 | INFO     | activation_logging.activations_logger:__init__:595 - JsonActivationsLogger.__init__ starting - output_dir: shared/goodwiki_jsonv2/activations.json, target_layers: all, sequence_mode: all, read_only: True
2026-01-20 12:46:35.907 | INFO     | activation_logging.activations_logger:__init__:603 - JsonActivationsLogger.__init__ - Setting up paths
2026-01-20 12:46:35.908 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:35.934 | INFO     | activation_logging.activations_logger:__init__:638 - JsonActivationsLogger.__init__ - Legacy metadata file exists, loading...
2026-01-20 12:46:46.234 | INFO     | activation_logging.activations_logger:__init__:644 - JsonActivationsLogger.__init__ - Legacy metadata loaded successfully
2026-01-20 12:46:46.553 | INFO     | activation_logging.activations_logger:__init__:665 - JsonActivationsLogger (Index+Sidecar format) initialized at shared/

Inference:   0%|          | 0/87 [00:00<?, ?it/s]

2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.979 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:46:59.

Inference:   0%|          | 0/150 [00:00<?, ?it/s]

2026-01-20 12:49:18.859 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.869 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.860 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.861 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.859 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.860 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.860 | INFO     | activation_logging.activations_logger:__init__:634 - JsonActivationsLogger.__init__ - Loading metadata
2026-01-20 12:49:18.

Inference: 100%|██████████| 150/150 [03:45<00:00,  1.50s/it]


In [21]:
res = embeddings 
df = ap_full.df.head(10000000)

for i, r in enumerate(res): 

    ishalu = df[df['prompt_hash'] == r['hashkey']]['halu']
    assert len(ishalu) == 1 
    res[i]['halu'] = ishalu.values

    




In [22]:
from sklearn.metrics import roc_auc_score

from activation_research.metrics import cosine_similarity_ood_stats, mahalanobis_ood_stats
    
mahalanobis_ood_stats(baseline_embeddings, res, outlier_class = 0)


{'mahalanobis_mean_id': 29.163002014160156,
 'mahalanobis_std_id': 5.990961074829102,
 'mahalanobis_mean_ood': 24.57122230529785,
 'mahalanobis_std_ood': 5.188235759735107,
 'mahalanobis_auroc': np.float64(0.7259842193617028)}

In [23]:


    
cosine_similarity_ood_stats(baseline_embeddings, res, outlier_class = 0)

{'cosine_mean_train': 0.9973577260971069,
 'cosine_std_train': 0.0008750715642236173,
 'cosine_mean_id': 0.9963846802711487,
 'cosine_std_id': 0.001229090616106987,
 'cosine_mean_ood': 0.9971453547477722,
 'cosine_std_ood': 0.0010400055907666683,
 'cosine_auroc': np.float64(0.6959588084496104)}